<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Papers/blob/main/DenseNet_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

tf.__version__

'2.11.0'

#Data 
[Source](https://www.kaggle.com/datasets/csafrit2/plant-leaves-for-image-classification)

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive'
!kaggle datasets download -d csafrit2/plant-leaves-for-image-classification
!unzip \*.zip && rm .zip

In [2]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory = '/content/Plants_2/train/',
    labels = 'inferred',
    label_mode = 'int', # categorical, binary
    class_names = None,
    color_mode = 'rgb', #grayscale
    batch_size = 32,
    image_size = (128, 128), #Reshape if not in this size
    shuffle=True,
    seed=32,
    validation_split=None,
    subset=None)

valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory = '/content/Plants_2/valid/',
    labels = 'inferred',
    label_mode = 'int', # categorical, binary
    class_names = None,
    color_mode = 'rgb', #grayscale
    batch_size = 32,
    image_size = (128, 128), #Reshape if not in this size
    shuffle=True,
    seed=32,
    validation_split=None,
    subset=None)


test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory = '/content/Plants_2/test/',
    labels = 'inferred',
    label_mode = 'int', # categorical, binary
    class_names = None,
    color_mode = 'rgb', #grayscale
    batch_size = 32,
    image_size = (128, 128), #Reshape if not in this size
    shuffle=True,
    seed=32,
    validation_split=None,
    subset=None)

Found 4274 files belonging to 22 classes.
Found 110 files belonging to 22 classes.
Found 110 files belonging to 22 classes.


In [3]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255,
    rotation_range=5,
    zoom_range = (0.95, 0.95),
    horizontal_flip=False,
    vertical_flip=False,
    data_format='channels_last', #channels_last
    validation_split=0.0,
    dtype=tf.float32
)

train_gen = train_datagen.flow_from_directory('/content/Plants_2/train/',
                                        target_size=(128, 128),
                                        batch_size=32,
                                        color_mode='rgb',
                                        class_mode='categorical', #binary, sparse
                                        shuffle=True,
                                        subset=None)


test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

validation_gen = test_datagen.flow_from_directory('/content/Plants_2/test/',
                                        target_size=(128, 128),
                                        batch_size=32,
                                        color_mode='rgb',
                                        class_mode='categorical', #binary, sparse
                                        shuffle=True,
                                        subset=None)

Found 4274 images belonging to 22 classes.
Found 110 images belonging to 22 classes.


In [4]:
keras.utils.plot_model(model, "my_first_model_with_shape_info.png", show_shapes=True)


In [4]:
train_ds.class_names

['Alstonia Scholaris diseased (P2a)',
 'Alstonia Scholaris healthy (P2b)',
 'Arjun diseased (P1a)',
 'Arjun healthy (P1b)',
 'Bael diseased (P4b)',
 'Basil healthy (P8)',
 'Chinar diseased (P11b)',
 'Chinar healthy (P11a)',
 'Gauva diseased (P3b)',
 'Gauva healthy (P3a)',
 'Jamun diseased (P5b)',
 'Jamun healthy (P5a)',
 'Jatropha diseased (P6b)',
 'Jatropha healthy (P6a)',
 'Lemon diseased (P10b)',
 'Lemon healthy (P10a)',
 'Mango diseased (P0b)',
 'Mango healthy (P0a)',
 'Pomegranate diseased (P9b)',
 'Pomegranate healthy (P9a)',
 'Pongamia Pinnata diseased (P7b)',
 'Pongamia Pinnata healthy (P7a)']

In [10]:
class DenseNet(tf.keras.Model):
  def __init__(self, x, n_blocks, filters, training):
    self.filters = filters
    self.training = training
    self.n_blocks = n_blocks
    self.model = Dense_net(x)

  def dense_block(self, input_x, n_layers, layer_name):
    with tf.name_scopet(layer_name):
      layers = list()

      layers.append(input_x)

      x = tf.keras.layers.BatchNormalization(axis=-1)(x)
      x = tf.keras.layers.Conv2D(filters=4*self.filters, kernel_size=1, activaion='relu')(x)
      x = 